In [19]:
PATH = "../data/dataset3/"
DATA_PATH = PATH + "TCGA-GBM_TCGA-THCA_TCGA-LAML_TCGA-HNSC_TCGA-LUAD_TCGA-UCEC_TCGA-KIRC_TCGA-SARC__GeneExpressionQuantification_HTSeq-Counts.csv"
SUBSET_PATH = PATH + "subsets/"

import numpy
data_file = open(DATA_PATH, "rb")

will load the data file and save into files per cancer type
please set PATH constants

In [3]:
data_file.seek(0)
gen_labels = numpy.genfromtxt(data_file, delimiter=",", max_rows=1, dtype=numpy.unicode)[1:]
gen_labels

array(['"ENSG00000000003"', '"ENSG00000000005"', '"ENSG00000000419"', ...,
       '"ENSG00000281910"', '"ENSG00000281912"', '"ENSG00000281920"'],
      dtype='<U17')

In [4]:
feature_count = gen_labels.shape[0]
feature_count
data_file.seek(0)
row_count = sum(1 for line in data_file)
row_count

3497

In [11]:
meta_file = open(DATA_PATH.replace(".csv","_metadata.csv"), "rb")
meta_data = numpy.genfromtxt(meta_file, delimiter=",", dtype=numpy.unicode)[:,1:]

data_file.seek(0)
data = numpy.empty((row_count-1, feature_count), dtype = numpy.float32)
cancer_labels = numpy.transpose([row.replace('"', '') for row in meta_data[1]]) #meta data has same order, so we just set the meta_data column 2 as label
for irow, line in enumerate(data_file):
    if irow == 0:
        continue
    row = line.split(b",")
    if meta_data[0,irow-1] != row[0].decode().replace('\n', ''):
        raise ValueError("bad Metadata")
    data[irow-1, :] = row[1:]
    # show progress
    if round(irow/float(row_count)*100) % 5 == 0 and round((irow+1)/float(row_count)*100) % 5 != 0:
        print(str(irow) + " (" + str(round(irow/float(row_count)*100)) + "%) rows parsed")


17 (0%) rows parsed
192 (5%) rows parsed
367 (10%) rows parsed
542 (15%) rows parsed
716 (20%) rows parsed
891 (25%) rows parsed
1066 (30%) rows parsed
1241 (35%) rows parsed
1416 (40%) rows parsed
1591 (45%) rows parsed
1765 (50%) rows parsed
1940 (55%) rows parsed
2115 (60%) rows parsed
2290 (65%) rows parsed
2465 (70%) rows parsed
2640 (75%) rows parsed
2815 (80%) rows parsed
2989 (85%) rows parsed
3164 (90%) rows parsed
3339 (95%) rows parsed


In [12]:
data_file.close()
meta_file.close()

## subsets

In [13]:
cancer_data = {}
for cancer_label in set(cancer_labels):
    cancer_data[cancer_label] = data[numpy.where(cancer_labels == cancer_label)[0][1:]]
cancer_data.pop('NA', None)

array([[  8.846632  ,   2.45225167,   9.77157307, ...,   2.45225167,
          7.56932259,   3.53934097],
       [ 10.24181175,   4.35493517,   9.929636  , ...,   2.45225167,
          6.65094805,   4.95146418],
       [  9.10805225,   4.18585443,  10.07141209, ...,   2.45225167,
          6.10550022,   4.34707355],
       ..., 
       [ 11.15729141,   6.38510513,  10.36147881, ...,   2.45225167,
          5.91513205,   3.45848203],
       [ 11.74471283,   3.38458467,  10.46324921, ...,   2.9967134 ,
          7.04641676,   3.21780109],
       [ 11.72798729,   3.43383503,  10.0963192 , ...,   2.45225167,
          6.30406713,   3.43383503]], dtype=float32)

In [17]:
import os
if not os.path.exists(SUBSET_PATH):
    os.makedirs(SUBSET_PATH)

numpy.save(SUBSET_PATH + "gen_labels", gen_labels)

for cancer_label, subset in cancer_data.items():
    numpy.save(SUBSET_PATH + cancer_label, subset)
